In [1]:
import os
from pydub import AudioSegment
import numpy as np

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Input, concatenate, Dropout, SpatialDropout1D, Flatten, BatchNormalization, Conv1D, MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
from clr_callback import CyclicLR

C:\Users\Ranet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Raw data preprocessing:

In [2]:
MAP_DIR_PATH = 'MAPS/PROTOTYPE MAPS/'
MAP_NAME = 'combined_maps'

In [3]:
# Load the beatmap hitcirlces

with open(os.path.join(MAP_DIR_PATH, MAP_NAME + '.txt')) as f:
    content = [x.strip().replace(',', ' , ').split(' ')[:5] + ['\n'] for x in f.readlines()]
    #content = [x.strip().split(',')[:3] for x in f.readlines()] # TRY WITH , AS WELL
for i in range(10):
    print(content[i])

['80', ',', '64', ',', '107', '\n']
['128', ',', '192', ',', '380', '\n']
['160', ',', '192', ',', '516', '\n']
['176', ',', '320', ',', '789', '\n']
['208', ',', '320', ',', '925', '\n']
['232', ',', '192', ',', '1198', '\n']
['264', ',', '192', ',', '1334', '\n']
['288', ',', '320', ',', '1607', '\n']
['320', ',', '320', ',', '1743', '\n']
['336', ',', '192', ',', '2016', '\n']


In [ ]:
import itertools
joined_maps = list(itertools.chain(*content))

length = 35
sequences = list()
for i in range(length, len(joined_maps)):
    # select sequence of tokens
    seq = joined_maps[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

for i in range(2):
    print(sequences[i])
    print('------------')
    
sequences = [' '.join(x) for x in sequences]
for i in range(2):
    print(sequences[i])
    print('------------')

Total Sequences: 1815441
['80', ',', '64', ',', '107', '\n', '128', ',', '192', ',', '380', '\n', '160', ',', '192', ',', '516', '\n', '176', ',', '320', ',', '789', '\n', '208', ',', '320', ',', '925', '\n', '232', ',', '192', ',', '1198', '\n']
------------
[',', '64', ',', '107', '\n', '128', ',', '192', ',', '380', '\n', '160', ',', '192', ',', '516', '\n', '176', ',', '320', ',', '789', '\n', '208', ',', '320', ',', '925', '\n', '232', ',', '192', ',', '1198', '\n', '264']
------------
80 , 64 , 107 
 128 , 192 , 380 
 160 , 192 , 516 
 176 , 320 , 789 
 208 , 320 , 925 
 232 , 192 , 1198 

------------
, 64 , 107 
 128 , 192 , 380 
 160 , 192 , 516 
 176 , 320 , 789 
 208 , 320 , 925 
 232 , 192 , 1198 
 264
------------


In [ ]:
label_vocab_size = 200000 # Don't really need more than 1000, but for some reason a bigger vocab_size is good for the tokenizer
label_seq_len = 10

def tokenization_processing(x_to_tok, vocab_size=1500, seq_len=2, pad=False):

    tokenizer = Tokenizer(num_words=vocab_size,
                                       filters='!"#$%&()*+-./:;<=>?@[\\]^_`{|}~\t',
                                       split=" ",
                                       char_level=False, # Could be very very interesting
                                       oov_token=None)

    tokenizer.fit_on_texts(x_to_tok)
    print('True vocab size:', len(tokenizer.word_index))
    
    X_tokenized = tokenizer.texts_to_sequences(x_to_tok)
    
    if pad:
        X_tokenized = pad_sequences(X_tokenized, maxlen=seq_len)
        X_val_tokenized = pad_sequences(X_val_tokenized, maxlen=seq_len)
    
    print('Before:', x_to_tok[0])
    print('After:', X_tokenized[0])
    
    return [X_tokenized, tokenizer]

Y_tokenized, tokenizer_y = tokenization_processing(sequences, vocab_size=label_vocab_size, seq_len=label_seq_len)
#X_train_tokenized, X_val_tokenized, tokenizer_x = tokenization_processing(X_train, X_val, vocab_size=feature_vocab_size, seq_len=feature_seq_len, pad=True)

True vocab size: 189734


In [ ]:
sequences = np.array(Y_tokenized)
X, y = sequences[:,:-1], sequences[:,-1]

#print(X[2])
#print(y[2])
print(X.shape, y.shape)

#X = X.reshape(X.shape[0], X.shape[1], 1)
#sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
#X = np.array(sequences)
#y = to_categorical(y, num_classes=vocab_size)

print(X[2], 'X')
print(y[2], 'y')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(y[0])
print(X[0])

In [22]:
# define model
model = Sequential()
model.add(Embedding(label_vocab_size, 50, input_length=length))
model.add(LSTM(64, recurrent_dropout=0.3))
model.add(Dense(label_vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 35, 50)            10000000  
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_2 (Dense)              (None, 200000)            13000000  
Total params: 23,029,440
Trainable params: 23,029,440
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
# compile model
#clr = CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000.)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X_train, y_train, batch_size=512, epochs=1000,
           validation_data=(X_val, y_val), verbose=1)#, callbacks=[clr])

Train on 1452352 samples, validate on 363089 samples
Epoch 1/1000
 279552/1452352 [====>.........................] - ETA: 1:58:23 - loss: 6.0970 - acc: 0.3332

KeyboardInterrupt: 

In [ ]:
#model.load_weights('MODEL/no_music_test_weights_2.h5')

model.fit(X_train, y_train, batch_size=256, epochs=1000,
           validation_data=(X_val, y_val), verbose=1)#, callbacks=[clr])

In [ ]:
from keras.models import load_model
model = load_model('MODEL/no_music_test_model_ruined2.h5')

In [ ]:
model.save('MODEL/no_music_test_model_ruined.h5')
model.save_weights('MODEL/no_music_test_weights_ruined.h5')
np.save('MODEL/no_music_test_ruined.np', model.get_weights())

In [ ]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        encoded = encoded.reshape(encoded.shape[0], encoded.shape[1], 1)
        # one hot encode
       # encoded = encoded, num_classes=len(mapping)
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# test start of rhyme
print(generate_seq(model, mapping, length, '''317,197,281
256,241,612
357,346,778
396,208,944
336,268,1109
195,197,1275
258,0,1607
374,80,1773
256,149,1938
292,74,2104
180,103,2270
101,259,2601
40,194,2933
203,238,3099
126,350,3265
298,265,3596
255,322,3928
336,200,4093''', 10000))